In [ ]:
# Uncomment the following commands to install in Colab
# Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment
# and then proceed to the following cells

# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.7b2 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch svg-figs
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from emutools.parameters import load_param_info
from aust_covid.inputs import get_ifrs
import arviz as az
from emutools.tex import DummyTexDoc, StandardTexDoc
from inputs.constants import SUPPLEMENT_PATH, RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN
import numpy as np
from aust_covid.plotting import plot_dispersion_examples, plot_targets
from aust_covid.utils import add_image_to_doc

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'dispersion', 'Target dispersion', 'austcovid')
dummy_doc = DummyTexDoc()
epi_model = build_model(dummy_doc, mobility_sens=True)
targets = get_targets(app_doc)
target_fig = plot_targets(targets)
caption = 'Calibration algorithm targets for likelihood calcuation with raw data from which they were derived.'
add_image_to_doc(target_fig, 'target_fig', 'svg', 'Calibration targets.', app_doc, 'Targets', caption=caption)
param_info = load_param_info()
prior_names = [p.name for p in get_priors(False, param_info['abbreviations'], dummy_doc)]
parameters = param_info['value'].to_dict() | get_ifrs(dummy_doc)
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

In [ ]:
colours = {'notifications_ma': '10, 10, 100', 'deaths_ma': '100, 10, 10'}
dispersion_fig = plot_dispersion_examples(idata, epi_model, parameters, prior_names, targets, colours, np.linspace(0.1, 0.9, 9))
dispersion_fig

In [ ]:
caption = 'Examples of the effect of values of the negative binomial distribution dispersion parameter, ' \
    'centiles of likelihood distribution. Actual targets used for likelihood calculation circles. '
add_image_to_doc(dispersion_fig, 'dispersion_examples', 'svg', 'Dispersion examples.', app_doc, 'Targets', caption=caption)
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex supplement
    ! biber supplement
    ! pdflatex supplement
    ! pdflatex supplement